In [37]:
from scipy.stats import norm, t
import numpy as np
import math
import statsmodels.api as sm 

In [6]:
n = 364 # Number of daily observations
save = {'mean': 2, 'sd': 0.1}
prod = {'mean': 20, 'sd': 5}
noise = {'mean': 0, 'sd': 5}
baseload = 50
intensity = 5

# Functions

In [7]:
def data_gen(n = 364, p = prod, ns = noise, s = None,  capital = None, period = 0):
    """Generates an array of observation [production, energy, period]
    Inputs:
    n = # of observations
    p = Object {mean, sd} modeling production as normal random variable
    ns = Object {mean, sd} modeling noise in energy data as normal random variable
    s = Object {mean, sd} modeling SEM savings as normal random variable
    capital = Object{magnitude, start} modeling capital project savings as a magnitude and 
        start date.  The start date is expressed in fraction of years (0.5 represents a project implemented half way through)
    period -> 0 = baseline, 1 = engagement
    
    Outputs:
    returns an array of the form [production, energy, savings] and the sum of savings"""
    production = np.random.normal(p['mean'], p['sd'], n) # generate production data
    # generate savings data
    if s is None:
        savings = [0 for _ in range(1, n)]
    else:
        savings = np.random.normal(s['mean'], s['sd'], n) # generate savings data
    
    # Generate capital project savings
    if capital is None:
        cap = [0 for _ in range (1, n)]
    else:
        cap = [capital['savings'] if capital['start'] > i/n else 0 for i in range(1,n)]
    
    noise = np.random.normal(ns['mean'], ns['sd'], n) # generate noise
    energy = [baseload + intensity* prod + noise - save - cap for prod, noise, save, cap in zip(production, noise, savings, cap)] 
    
    # calculate energy data -> energy = baseload + intensity*production + noise - savings
    sem_savings = sum(savings)
    cap_savings = sum(cap)
    data = [[production, energy, period] for production, energy in zip(production, energy)]
    return data, sem_savings, cap_savings

In [8]:
def forecast_mean_pi(data):
    """Calculate predicted and actual energy consumption and prediction standard error 
    Inputs:
        data:  array baseline and engagement observations in the form [production, energy, period]
    Returns: 
        predicted_kwh:  predicted engagement period kWh using baseline model
        actual_kwh: actual enagement period kWh
        pred_se: standard error of predicted_kwh.  
            Calculated as the square root of the sum of two variance components.
                [1] The variance of the estimate of the expected value of annual consumption. Calculated using the
                Convariance Matrix.
                [2] The sum over the entire year of the variance of the noise in the meter-period-level observations.
        """
    # Filter baseline data and fit model.
    X = [x[0] for x in data if x[2] == 0] # X is baseline production data
    Y = [y[1] for y in data if y[2] == 0] # Y is baseline energy data
    X = sm.add_constant(X) # Adds intercept to model
    lm = sm.OLS(Y, X) # Form of model
    results = lm.fit() # fits model
    
    # Calulate predicted annual load (kWh)
    engagement = [x for x in data if x[2] == 1] # Filter to engagement period
    beta = results.params #Beta 
    predict_kwh = sum([[1, x[0]] @ beta for x in engagement]) # need to add constant
    actual_kwh = sum([x[1] for x in engagement])
    
    #Calculate the mean variance
    n = len(data)
    vcov = results.cov_params()
    mse_resid = results.mse_resid
    mean_var = sum([[1, x[0]] @ vcov @ [1, x[0]] for x in data])
    noise_var = n * mse_resid
    pred_se = math.sqrt(mean_var + noise_var)
    
    return predict_kwh, actual_kwh, pred_se

In [9]:
def pre_post_ci(data):
    """Calculates pre/post model savings and standard error
    
    Inputs:
        data: array of baseline and engagement observations in the form [production, energy, period]
        
    Output:
        save: total (SEM and capital) energy savings 
        save_se: total energy savings standard error """
    
    # Format data for model
    X = [[x[0], x[2]] for x in data]  # X is in format [production, period]
    X = sm.add_constant(X)
    Y = [y[1] for y in data]
    lm = sm.OLS(Y,X)
    results = lm.fit()
    
    # Calculate the # of observations in the engagement period
    n = len([x for x in data if x[2] ==1])
    
    # Calculate ci for period indicator
    save = -1*results.params[2] * n
    save_se = results.bse[2] * n
    
    return save, save_se

In [81]:
def simulate(n = 100, sv = save, cap = None):
    """Runs simulation
        [1] Generates simulated data
        [2] Calculates energy saved by forecast and pre/post method
        [3] Compares actual savings to 80% CI of estimated savings
        [4] Calculates average 80% CI of estimated savings over trail runs
    
    Inputs:
        n:  # of simulation trails
        sv: Object {mean, sd} to model savings as normal random variable
        cap:  Object {savings, start} modeling the implementation of a capital project
    
    Output:
        success rate forecast prediction ci
        success rate pre/post ci
        forecast_ci (upr and lwr): average forecast prediction upr and lwr 80% CI over trails
        prepost_ci (upr and lwr):  average prepost predicition upr and lwr 80% CI over trails
        
        The success rate is # trails with ci / # trails
        Where ci = est +/- 1.282 * est_se """
    
    # initialize within to count # of actual savings within the model's 80% confidence interval
    # initialize mds to count # of models checked
    within_f, within_pp = 0,0
    forecast_ci = (0,0)
    prepost_ci = (0,0)
    mds = 0

    for _ in range(n):
        # Generate Baseline Data
        baseline, _, _ = data_gen() # Generate baseline data 
        engagement, save_actual, cap_actual = data_gen(s = sv, capital = cap, period = 1) # Generate engagement data
        data = [*baseline, *engagement] # Combine baseline and engagement data
    
        # FORECAST - Model and Results
        predict_kwh, actual_kwh, pred_se = forecast_mean_pi(data)
        save_model = predict_kwh - actual_kwh - cap_actual # Calculated the forecast model estimate of savings
    
        # FORECAST - Comparison to actual
        upr, lwr = save_model + 1.28 * pred_se, save_model - 1.28 * pred_se
        if lwr <= save_actual <= upr: within_f += 1
        run_lwr, run_upr = forecast_ci #add loop results to running total
        forecast_ci = run_lwr + lwr, run_upr + upr
        
        #PREPOST - Model and Results
        pp_all, pp_se = pre_post_ci(data)
        pp_sem = pp_all - cap_actual # Remove capital project savings
        
        # PREPOST - Comparison to actual 
        upr, lwr = pp_sem + 1.28 * pp_se, pp_sem - 1.28 * pp_se
        if lwr <= save_actual <= upr: within_pp += 1
        run_lwr, run_upr = prepost_ci #add loop results to running total
        prepost_ci = run_lwr + lwr, run_upr + upr
            
        mds +=1
    
    return within_f/mds*100, within_pp/mds*100, (ci/mds for ci in forecast_ci),(ci/mds for ci in prepost_ci)

# Simulations

Simulations are run for various combinations of energy savings, energy savings variability, and with or without capital project.  

1. __Energy Savings__ 

    1. Mean value ranging for low (2 - 1.3%), medium (5 - 3.3%), and high (10 - 6.6%)
    2. Variablity ranging from no variability (sd = 0.01) to high variability (sd = 50% mu)
    
2. __Capital Projects:__ Two Scenarios are considered   

    1. No capital projects
    2. 1 medium capital project (3.3%) implemented midway through the engagement period.
    
__Run 2,500 trials:__
    
1. Generate data.  Record the actual savings value.
2. Fit forecast model and pre/post model.
3. Compared actual savings value to the models' 80% CI's
    
__Compute Summary Statistics:__
 
1. Calculate frequency of actual savings within model CI's
2. Calculate average CI's

In [86]:
# Initialize Simulation
sim_results = []
savings_models = [{'mean':2, 'sd':0.1}, {'mean':2, 'sd':1},
                  {'mean':5, 'sd':2.5}, {'mean':10, 'sd':5}]
capital_models = [None, {'savings':5, 'start':0.5}]

# Run Simulations
for sv, cap in [(sv, cap) for sv in savings_models for cap in capital_models]:
    
    forecast, prepost, forecast_ci, prepost_ci = simulate(2500, sv, cap)
    capital = 'X' if cap else '-'
    sim_results.append([sv['mean'], sv['sd'], capital ,forecast, prepost, *forecast_ci, *prepost_ci])

# Print Result
print("{:^8} {:^8} {:^8} {:^8} {:^8} {:^16} {:^16}".format('', 'savings', '',
                                       'Forecast', 'Prepost', 
                                        'Forecast', 'Pre/Post'))
print("{:^8} {:^8} {:^8} {:^8} {:^8} {:^8} {:^8} {:^8} {:^8}".format('savings', 'variance', 'capital',
                                       '% Correct', '% Correct', 
                                        'lwr', 'upr', 'lwr', 'upr'))
    
for row in sim_results:
    print("{:^8} {:^8} {:^8} {:>8.2f}% {:>8.2f}% {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f}".format(*row))
    

         savings           Forecast Prepost      Forecast         Pre/Post    
savings  variance capital  % Correct % Correct   lwr      upr      lwr      upr   
   2       0.1       -        79.96%    80.16%   552.01   897.63   552.35   897.39
   2       0.1       X        80.72%    83.68%   555.33   900.60   544.79   910.75
   2        1        -        78.92%    79.52%   557.99   903.97   556.49   905.14
   2        1        X        80.16%    83.28%   555.96   901.82   544.10   913.75
   5       2.5       -        80.40%    82.72%  1640.96  1986.71  1630.75  1996.79
   5       2.5       X        81.60%    86.28%  1642.35  1987.88  1622.23  2008.11
   10       5        -        80.12%    88.00%  3456.59  3801.98  3417.58  3840.50
   10       5        X        78.96%    89.00%  3454.31  3799.82  3406.85  3846.97


## Interpreting Simulation Results

The forecast method builds a model on only the baseline data.  The pre/post method incorporates data from both the baseline and engagement period.  When the signal the model is attempting to detect (energy savings) varies, the signal variation increases standard error in the pre / post method result. 

__Comparing the forecast and the pre/post results__

_When energy savings are consistent_, the forecast and pre / post method yield similar standard errors.
_When energy savings vary_, the forecast method yields a smaller standard error than the pre / post method.
   
The results indicate that the forecast method is robust to variability in energy savings.  The pre / post method is sensitive to variation in SEM savings and capital projects.
   
__Does it matter?__
Probably.  SEM is designed to create variable energy savings.  SEM promotes:

1. Incremental improvements (savings  increase over time)
2. Capital projects (SEM participants triple the rate of capital project implementation)
3. Targeted improvements (i.e. energy efficiency measures than apply to only specific operating conditions)

As a result, the most extreme scenario modeled (high savings, high variance, and capital project) is not extreme nor uncommon in the SEM portfolio.